In [1]:
import os, json
from pathlib import Path, PureWindowsPath # for Unix compatibility

from cri.robot import SyncRobot
from cri.controller import Mg400Controller as Controller
from vsp.video_stream import CvImageOutputFileSeq, CvVideoDisplay, CvPreprocVideoCamera   
from vsp.processor import CameraStreamProcessor, AsyncProcessor

from tactile_servoing_2d.lib.common import home, control
from pose_models_2d.lib.models.cnn_model import CNNmodel as Model

data_path = os.path.join(os.environ["DATAPATH"], "open", "tactile-servoing-2d-dobot")

In [2]:
# User-defined paths
test_dir = os.path.join("digitac", "servo_edge2d", "new") 
model_dir = os.path.join("digitac", "model_edge2d", "train", "train2d_cnn")  

# Open saved meta dictionaries
with open(os.path.join(data_path, model_dir, "meta.json"), 'r') as f: 
    model_meta = json.load(f)

# Make the new meta dictionary
meta = {**model_meta, 
    # ~~~~~~~~~ Paths ~~~~~~~~~#
    "meta_file": os.path.join(test_dir, "meta.json"),        
    "test_image_dir": os.path.join(test_dir, "frames_bw"), # to store tactile images
    "test_df_file": os.path.join(test_dir, "predictions.csv"), # to store predictions and robot poses
    # ~~~~~~~~~ Robot movements ~~~~~~~~~#
    "home_pose": [300, 0, -90, 0, 0, -90],  
    "linear_speed": 20,
    "num_steps": 400, # preset number of servo control steps
    # ~~ disk ~~
    # "work_frame": [283, 0, -100-5, 0, 0, -90], # TacTip
    "work_frame": [287, 0, -112-4, 0, 0, -90-8], # DigiTac
    # "work_frame": [287, 0, -118-4.5, 0, 0, -90-8], # Digit
    # ~~ flower ~~
    # "work_frame": [278, 0, -100-5, 0, 0, -90], # TacTip
    # "work_frame": [282, 0, -112-4, 0, 0, -90-8], # DigiTac
    # "work_frame": [282, 0, -118-4.5, 0, 0, -90-8], # Digit
    # ~~ square ~~
    # "work_frame": [288, 0, -100-5, 0, 0, -90], # TacTip
    # "work_frame": [292, 0, -112-4, 0, 0, -90-8], # DigiTac
    # "work_frame": [292, 0, -118-4.5, 0, 0, -90-8], # Digit
    # ~~ other settings ~~
    "source": 0, # which usb camera (0,1,2) CHECK IF CODE FAILS 
    "r": [1, 0, 0, 0, 0, 0], # set point of controller (also used to make fixed steps)
    "kp": [1, 0.5, 0, 0, 0, 0.5], # p-control gains 
    "ki": [0, 0.3, 0, 0, 0, 0.1], # i-control gains
    "ei_bnd": [[5, 0, 0, 0, 0, 45], [-5, 0, 0, 0, 0, -45]], # windup bounds 
    # ~~~~~~~~~ Comments ~~~~~~~~~#
    "comments": "servo control with 2d edge on MG400"
    }

In [3]:
# Save dictionary to file
os.makedirs(os.path.join(data_path, test_dir), exist_ok=True)
with open(os.path.join(data_path, meta["meta_file"]), 'w') as f:
    json.dump(meta, f)

# Absolute posix paths
for key in [k for k in meta.keys() if "file" in k or "dir" in k]:
    meta[key] = os.path.join(data_path, meta[key])
    meta[key] = Path(PureWindowsPath(meta[key])).as_posix() # for Unix

In [4]:
def make_sensor(size, crop, exposure, source, threshold=None, **kwargs):  # amcap: reset all settings; autoexposure off; saturdation max
    camera = CvPreprocVideoCamera(size, crop, threshold, exposure=exposure, source=source)
    for _ in range(5): camera.read() # Hack - camera transient   
    return AsyncProcessor(CameraStreamProcessor(camera=camera,
            display=CvVideoDisplay(name='sensor'),
            writer=CvImageOutputFileSeq()))

In [5]:
# Startup/load model 
model = Model()
model.load_model(**meta)

# Control robot with loaded model
with SyncRobot(Controller()) as robot, make_sensor(**meta) as sensor:       
    home(robot, **meta)     
    pred, _ = control(robot, sensor, model, **meta)  

# Save run to file
pred.to_csv(meta["test_df_file"], index=False)

(MainThread) Model was constructed with shape (None, 160, 120, 1) for input Tensor("input:0", shape=(None, 160, 120, 1), dtype=float32), but it was called on an input with incompatible shape (None, 120, 160, 1).


1/1 [==============================] - 0s 972us/step
1: v=[ 1 0 0-0 0-0.]
     y=[ 0   0.17  0   0   0  -0.45]
joints=[ -7.73  54.92  68.72 -20.45]
1/1 [==============================] - 0s 3ms/step
2: v=[ 2  -0.08  0  -0   0   0.27]
     y=[ 0   0.11  0   0   0  -1.23]
joints=[ -7.73  54.92  68.72 -20.45]
1/1 [==============================] - 0s 3ms/step
3: v=[ 3  -0.12  0  -0   0   1.05]
     y=[ 0  -2.67  0   0   0   6.68]
joints=[ -7.73  54.92  68.72 -20.45]
1/1 [==============================] - 0s 3ms/step
4: v=[ 3.97  1.17  0   0   0  -2.79]
     y=[ 0  -2.75  0   0   0   7.17]
joints=[ -7.73  54.92  68.72 -20.45]
1/1 [==============================] - 0s 3ms/step
5: v=[ 5.02  2.44  0   0   0  -7.59]
     y=[ 0  -2.49  0   0   0  -8.32]
joints=[ -7.73  54.92  68.72 -20.45]
1/1 [==============================] - 0s 3ms/step
6: v=[ 6.18  3.61  0   0   0  -3.82]
     y=[  0   -1.82   0    0    0  -13.58]
joints=[ -7.73  54.92  68.72 -20.45]
1/1 [==============================] - 0